In [1]:
with open("article_title.txt", "r") as f:
    paper_list = f.readlines()

paper_list = [i.strip().replace("\xa0", " ") for i in paper_list if i.strip()]
paperList = list(set(paper_list))
print(len(paperList))
paperList = sorted(paperList, key=lambda x:paper_list.index(x))

11


In [2]:
import os
from collections import defaultdict

from selenium import webdriver

In [3]:
download_dir = "pdf_downloaded"

if not os.path.exists(download_dir):
    os.mkdir(download_dir)

In [11]:
profile = webdriver.FirefoxProfile()
profile.set_preference('permissions.default.stylesheet', 2)
profile.set_preference('permissions.default.javascript', 2)
profile.set_preference('permissions.default.image', 2)
profile.set_preference('browser.download.folderList', 2)
profile.set_preference('browser.download.dir', os.path.abspath(download_dir))
profile.set_preference('browser.download.manager.showWhenStarting', False)
profile.set_preference('browser.helperApps.neverAsk.saveToDisk', 'application/pdf')
profile.set_preference('pdfjs.disabled', True)

browser = webdriver.Firefox(firefox_profile=profile)

In [5]:
total_di = defaultdict(defaultdict)
pubmed_url_base = "https://www.ncbi.nlm.nih.gov/pubmed/"

def similar_results_judge():
    try:
        item = browser.find_element_by_class_name("result_count")
        if int(item.text.split()[-1]) >1:
            print(item.text)
            try:
                correct_paper = browser.find_element_by_class_name("sensor_content")
                correct_paper.find_elements_by_tag_name("a")[-1].click()
                print("`sensor_content` found! Jumping to the detailed page..." )
            except:
                print("`sensor_content` not found. Trying again using simple mode..." )
                return True
    except:
        pass

In [6]:
def search_pubmed(i, a, di, title_mode=True):
    # search papers in pubmed
    if title_mode:
        pubmed_ = pubmed_url_base + "?term=" + a + "[Title]"
    else:
        pubmed_ = pubmed_url_base + "?term=" + a
    
    #browser.set_page_load_timeout(100)
    try:
        browser.get(pubmed_)
        
        if similar_results_judge():
            # trying searching without specifying the '[Title]' constrain
            search_pubmed(i, a, di, title_mode=False)

        di[i]['title'] = a

        cit = browser.find_element_by_class_name("cit")
        di[i]['desc'] = cit.text
        di[i]['journal'] = cit.find_element_by_tag_name('a').text

        aux =  browser.find_element_by_class_name("aux")
        dt = aux.find_elements_by_tag_name("dt")
        dd = aux.find_elements_by_tag_name("dd")
        
        for ii,v in enumerate(dt):
            di[i][v.text] = dd[ii].text
            
        abstr = browser.find_element_by_class_name("abstr")        
        di[i]['abstr'] = abstr.text

        #print(di[i])
    except Exception as e:
        print("Error!",e)
    finally:
        print("processed %s"%i)
        print("-"*20)

In [29]:
for i,a in enumerate(paperList):
    search_pubmed(i, a, total_di)
    
max_iter = 5
iter = 0
while iter < max_iter:
    rest = set(range(len(paperList))) - set(total_di.keys())
    if not rest:
        break
    print(rest)
    for i in rest:
        search_pubmed(i, paperList[i], total_di)
    iter +=1

processed 0
--------------------
Error! Message: Timeout loading page after 300000ms

processed 1
--------------------
Error! Message: Timeout loading page after 300000ms

processed 2
--------------------
Error! Message: Timeout loading page after 300000ms

processed 3
--------------------
processed 4
--------------------
processed 5
--------------------
processed 6
--------------------
Error! Message: Timeout loading page after 300000ms

processed 7
--------------------
Items: 1 to 20 of 287
`sensor_content` not found. Trying again using simple mode...
Items: 1 to 20 of 287
`sensor_content` found! Jumping to the detailed page...
processed 8
--------------------
processed 8
--------------------
processed 9
--------------------
Error! Message: Timeout loading page after 300000ms

processed 10
--------------------
{10, 3, 7}
Error! Message: Timeout loading page after 300000ms

processed 10
--------------------
processed 3
--------------------
Error! Message: Timeout loading page after 30

In [84]:
ncbi_url = "https://www.ncbi.nlm.nih.gov/"

def download_pmc_pdf(di, pmcid_key):
    
    for i,d in di.items():
        if pmcid_key in d.keys():
            
            pmc_url = ncbi_url + "pmc/articles/" + d[pmcid_key] + "/"

            browser.get(pmc_url)
            try:
                down_ = browser.find_element_by_class_name("format-menu")
                pdf_link = down_.find_elements_by_tag_name("a")[2]
                # starting to download pdf
                pdf_link.click()

                pdf_url = pdf_link.get_attribute('href')
                filename = pdf_url.split('/')[-1]
                status = os.path.exists(filename)

                di[i]['pdf_url'] = pdf_url
                di[i]['filename'] = filename if status else ""

            except Exception as e:
                print(e)
                
download_pmc_pdf(total_di, 'PMCID:')

In [31]:
for k,d in total_di.items():
    print(k, d.keys())

0 dict_keys(['title', 'desc', 'journal', 'PMID:', 'abstr'])
1 dict_keys(['title', 'desc', 'journal', 'PMID:', 'abstr'])
2 dict_keys(['title', 'desc', 'journal', 'PMID:', 'DOI:', 'abstr'])
4 dict_keys(['title', 'desc', 'journal', 'PMID:', 'DOI:', 'abstr'])
5 dict_keys(['title', 'desc', 'journal', 'PMID:', 'DOI:', 'abstr'])
6 dict_keys(['title', 'desc', 'journal', 'PMID:', 'DOI:', 'abstr'])
8 dict_keys(['title', 'desc', 'journal', 'PMID:', 'DOI:', 'abstr'])
9 dict_keys(['title', 'desc', 'journal', 'PMID:', 'PMCID:', 'DOI:', 'abstr'])
3 dict_keys(['title', 'desc', 'journal', 'PMID:', 'DOI:', 'abstr'])
10 dict_keys(['title', 'desc', 'journal', 'PMID:', 'DOI:', 'abstr'])
7 dict_keys(['title', 'desc', 'journal', 'PMID:', 'abstr'])


In [33]:
journal_li = []
for k,d in total_di.items():
    downloaded = 'filename' in d.keys()
    print(k, end=' '*5)
    #print(d['journal'], end=' '*5)
    journal_li.append(d['journal'])
    print(downloaded, end=' '*5)
    if not downloaded:
        print(d['DOI:'] if 'DOI:' in d.keys() else "No doi!", end="")
    print("\n")

# https://doi.org/

0     False     No doi!

1     False     No doi!

2     False     10.1016/j.jdermsci.2006.09.008

4     False     10.2165/00128071-200304120-00004

5     False     10.1111/j.1600-0846.2007.00275.x

6     False     10.1111/ijd.12741

8     False     10.1111/j.1525-1470.2006.00191.x

9     False     10.1194/jlr.P030338

3     False     10.1126/science.139.3556.766

10     False     10.1016/j.jmbbm.2013.04.008

7     False     No doi!



In [103]:
set(journal_li)

{'Ann Intern Med.',
 'Biol Psychiatry.',
 'Circulation.',
 'Diabetes Care.',
 'Eur Heart J Cardiovasc Imaging.',
 'Eur Heart J.',
 'Gut.',
 'J Am Coll Cardiol.',
 'JACC Cardiovasc Imaging.'}

In [125]:
check_if_url = "http://www.greensci.net/search?kw="

for i in set(journal_li):
    kw = i.rstrip('.').replace(' ', '+')
    print(check_if_url+kw)
    browser.get(check_if_url+kw)
    browser.save_screenshot("img/"+kw)
    ts(10)

http://www.greensci.net/search?kw=J+Am+Coll+Cardiol


/usr/local/lib/python3.6/dist-packages/selenium/webdriver/remote/webdriver.py:1031: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  "type. It should end with a `.png` extension", UserWarning)


http://www.greensci.net/search?kw=Eur+Heart+J
http://www.greensci.net/search?kw=Ann+Intern+Med
http://www.greensci.net/search?kw=Circulation
http://www.greensci.net/search?kw=Eur+Heart+J+Cardiovasc+Imaging
http://www.greensci.net/search?kw=Biol+Psychiatry
http://www.greensci.net/search?kw=Diabetes+Care
http://www.greensci.net/search?kw=Gut
http://www.greensci.net/search?kw=JACC+Cardiovasc+Imaging


In [133]:
table = browser.find_element_by_class_name("result-table-data")
tr = table.find_elements_by_tag_name("tr")[1:]

In [134]:
[td.text for td in tr[0].find_elements_by_tag_name("td")]

['1936-878X',
 'JACC-CARDIOVASC IMAG\nJACC-CARDIOVASCULAR IMAGING',
 '7.815',
 '10.189',
 '10.247']

In [10]:
browser.quit()

In [115]:
import pickle
with open("papers_info.pickle", "wb") as f:
    pickle.dump(total_di, file=f)

In [35]:
paperList

['Females have lower skin surface pH than men. A study on the surface of gender, forearm site variation, right/left difference and time of the day on the skin surface pH.',
 'In vitro transepidermal water loss: differences between black and white human skin.',
 'In vivo data of epidermal thickness evaluated by optical coherence tomography: effects of age, gender, skin type, and anatomic site.',
 'Racial Differences in Skin Resistance.',
 'Racial (ethnic) differences in skin properties: the objective data.',
 'Relationships between skin properties and environmental parameters.',
 'Seasonal variability in the biophysical properties of forehead skin in women in Guangzhou City, China.',
 'Seasonal variability in the biophysical properties of stratum corneum from different anatomical sites.',
 'Impairment of skin barrier function is not inherent in atopic dermatitis patients: a prospective study conducted in newborns.',
 'Increase in short-chain ceramides correlates with an altered lipid or

In [46]:
total_di[10]

defaultdict(None,
            {'title': 'In vivo skin biophysical behaviour and surface topography as a function of ageing.',
             'desc': 'J Mech Behav Biomed Mater. 2013 Dec;28:474-83. doi: 10.1016/j.jmbbm.2013.04.008. Epub 2013 Apr 17.',
             'journal': 'J Mech Behav Biomed Mater.',
             'PMID:': '23664827',
             'DOI:': '10.1016/j.jmbbm.2013.04.008',
             'abstr': "Abstract\nNormal skin ageing is characterised by an alteration of the underlying connective tissue with measurable consequences on global skin biophysical properties. The cutis laxa syndrome, a rare genetic disorder, is considered as an accelerated ageing process since patients appear prematurely aged due to alterations of dermal elastic fibres. In the present study, we compared the topography and the biomechanical parameters of normal aged skin with an 17 year old cutis laxa patient. Skin topography analyses were conducted on normal skin at different ages. The results indicate tha